In [13]:

import awswrangler  as wr
import pandas as pd
import pymysql
import datetime
import requests
import json

# def lambda_handler(event, context):
sql = """
with log_raw as (
select *
, case when user_agent like '%iPhone%' then 1 else 0 end as is_iphone
from moyo_logs.user_behavior_logs
where 1=1
--     and date_ymd >= '2023-09-09'
    and date_ymd >= '2023-06-10'
    and date_ymd < '2023-09-12' -- last 90일
-- limit 100
)
, iphone as (
select
distinct user_id
from log_raw
where 1=1
    and is_iphone = 1
)
, User_agg as (
select
user_id
, min( client_access_time) as first_time
, max( client_access_time) as last_time
, count(*) as cnt
, count( case when category = 'apply'
            and object_type = 'plan' then 1 end ) as apply_cnt
from log_raw
where 1=1
group by 1
)
, rel_page as (
select
distinct user_id
from log_raw a
where 1=1
    and (a.navigation = 'search_phones'
        or    a.navigation = 'phone_contents'
        or  a.navigation = 'contents_detail'
        )
)
, iphone15 as (
select
distinct  user_id
from moyo_data."signup_event_meta"
where 1=1
    and event_type = 'pre_iphone15_register'
group by 1
)
, stock as (
select
distinct user_id
from moyo_logs."amplitude_logs"
where 1=1
    -- and date_ymd  = '2022-06-09'
    and event_type =  'Click_알림받기_재고알리미'
-- group by 1 -- 명목 < 실질
    and user_id is not null
-- group by 1
)
, raw as (
select a.*
, d.profile
, d.phone_number
,  REPLACE(REPLACE(REPLACE(d.phone_number, '+82 ', '0'), '-', ''), ' ', '') as formatted_phone
, case when b.user_id is not null then 1 else 0 end as is_iphone
, case when c.user_id is not null then 1 else 0 end as is_iphone15
, case when e.user_id is not null then 1 else 0 end as is_rel_page
, e.user_id as stock_user_id
, case when f.user_id is not null then 1 else 0 end as is_stock
, marketing_consent
from user_agg  a
join moyo_data.moyo_user_v2 d on d.id = a.user_id
left join iphone b on a.user_id = b.user_id
left join iphone15 c on a.user_id = c.user_id
left join rel_page e on a.user_id = e.user_id
full outer join stock f on a.user_id = cast ( f.user_id as integer )
where 1=1
--      and d.marketing_consent = 1
)
select
coalesce(user_id ,stock_user_id ) as user_id
, phone_number
, formatted_phone
, profile
, first_time
, last_time
, cnt
, apply_cnt
, is_iphone
,  is_iphone15
, is_rel_page
, is_stock
, marketing_consent
from raw
where 1=1


"""

df = wr.athena.read_sql_query(sql=sql, database="moyo_order")
# bucket = 'moyo-mart'
# table_name = 'paid_ads_card'

# res = wr.s3.to_parquet(
#     df=df,
#     path=f"s3://{bucket}/{table_name}/",
#     dataset=True,
#     database="moyo_mart",
#     table=table_name, 
#     mode="overwrite", 
#     partition_cols=['date_ymd']
# )


# # https://s3.console.aws.amazon.com/s3/buckets/moyo-mart?region=ap-northeast-2&prefix=ringo_partition_test/&showversions=false
# sql_query = """

# """

# df = wr.athena.read_sql_query(sql=sql_query, database="moyo_logs")

# bucket = 'moyo-mart'
# table_name = 'huchat_crm_230912'

# res = wr.s3.to_parquet(
#     df=df,
#     path=f"s3://{bucket}/{table_name}/",
#     dataset=True,
#     database="moyo_mart",
#     table=table_name,
#     mode="overwrite_partitions",
#     partition_cols=['append']
# )

In [14]:
df.head()


,user_id,phone_number,formatted_phone,profile,first_time,last_time,cnt,apply_cnt,is_iphone,is_iphone15,is_rel_page,is_stock,marketing_consent
0,<NA>,<NA>,<NA>,<NA>,NaT,NaT,<NA>,<NA>,0,0,0,1,<NA>
1,<NA>,<NA>,<NA>,<NA>,NaT,NaT,<NA>,<NA>,0,0,0,1,<NA>
2,<NA>,<NA>,<NA>,<NA>,NaT,NaT,<NA>,<NA>,0,0,0,1,<NA>
3,<NA>,<NA>,<NA>,<NA>,NaT,NaT,<NA>,<NA>,0,0,0,1,<NA>
4,<NA>,<NA>,<NA>,<NA>,NaT,NaT,<NA>,<NA>,0,0,0,1,<NA>


In [15]:
import numpy as np

df['ab'] = np.random.choice(['a', 'b'], size=len(df))


In [16]:
bucket = 'moyo-mart'
table_name = 'huchat_crm_230912'

res = wr.s3.to_parquet(
    df=df,
    path=f"s3://{bucket}/{table_name}/",
    dataset=True,
    database="moyo_mart",
    table=table_name,
    mode="overwrite",
    # partition_cols=['append']
)

In [9]:
df

,user_id,phone_number,formatted_phone,profile,first_time,last_time,cnt,apply_cnt,is_iphone,is_rel_page,is_stock,ab
0,251169,+1 484-587-6802,+14845876802,나린,2023-06-11 02:35:26,2023-06-11 03:07:30,260,0,1,0,0,b
1,208265,+82 10-9633-6973,01096336973,구웅서,2023-06-13 00:13:02,2023-06-18 09:23:18,322,0,0,0,0,a
2,251117,+82 10-9024-5725,01090245725,박비채,2023-06-11 00:28:09,2023-06-11 15:52:50,2610,5,0,1,0,a
3,251192,+82 10-2447-7363,01024477363,지원,2023-06-11 04:22:39,2023-07-12 11:26:51,460,0,0,0,0,a
4,205151,+82 10-7754-9232,01077549232,최형호(Daniel),2023-06-10 02:15:02,2023-08-31 22:19:00,2453,3,0,1,0,b
...,...,...,...,...,...,...,...,...,...,...,...,...
89838,235861,+82 10-3357-5407,01033575407,손정학,2023-08-22 23:08:45,2023-09-11 23:53:05,1066,4,0,1,0,a
89839,343081,+82 10-5315-2674,01053152674,권동관,2023-08-31 14:52:39,2023-09-10 16:48:07,1059,1,0,0,0,b
89840,334432,+82 10-6844-0513,01068440513,전아윤,2023-08-22 14:00:59,2023-08-23 11:04:20,183,0,0,0,0,a
89841,119149,+82 10-8422-4948,01084224948,연수인,2023-08-01 08:52:39,2023-08-05 18:54:45,302,2,1,0,0,a


In [ ]:
Index(['관리코드', '레벨', '집계', 'Month', 'Value', '관리계정명', 'Source', '상세설명',
       'name_5_raw', 'name_4_raw', 'name_3_raw', 'name_2_raw', 'name_1_raw',
       'code', 'name_10', 'description', 'name_5', 'name_4', 'name_3',
       'name_2', 'name_1', 'is_plus', 'lv'],
      dtype='object')

In [7]:
sql = """
with order_raw as (
select
a.id as order_id
, a.reg_ts
, a.completed_at
, a.requested_at
, a.mvno
, a.mno as net
, a.plan_name
, a.plan_id -- 2022-04-27일 이후로만 사용가능 (plan_history_id로 대체)
, a.plan_history_id
, case when a.completed_at is not null then 1 else 0 end as is_compeleted
, a.order_type
, CASE
    WHEN a.status IN ('APPLICATION_CANCELED', 'APPLICATION_REJECTED', 'ACTIVATION_CANCELED')
    THEN 'canceled'
    WHEN a.status IN ('ACTIVATION_COMPLETE', 'ACTIVATION_SELF_COMPLETE', 'SELF_COMPLETE')
            AND a.order_type = 'self'
            AND a.activation_method = '유저_원통'
    THEN 'user_one' --
    when a.status in ('ACTIVATION_COMPLETE','ACTIVATION_SELF_COMPLETE', 'SELF_COMPLETE')
        and order_type = 'self' and (activation_method IS NULL or activation_method = '모요_셀프개통_서비스')
    then 'self' -- 모요 셀프개통
    when a.status in ('ACTIVATION_COMPLETE','ACTIVATION_SELF_COMPLETE', 'SELF_COMPLETE')
                and order_type = 'self' and activation_method  = '자동_개통'
    then 'auto' --자동개통
    when a.status in ('ACTIVATION_COMPLETE','ACTIVATION_SELF_COMPLETE', 'SELF_COMPLETE')
            and order_type = 'register' and activation_method is null
    then 'call' -- 상담사개통
    when a.status in ('ACTIVATION_COMPLETE','ACTIVATION_SELF_COMPLETE', 'SELF_COMPLETE')
                and order_type = 'register' and activation_method = '파트너스_원통'
    then 'partner_one'
    else 'etc'
END AS order_type_detail
,1.00* date_diff( 'minute', a.reg_ts, a.completed_at)/60 as diff_hour_start_comp
,1.00* date_diff( 'minute', a.reg_ts, a.requested_at)/60 as diff_hour_start_req
,1.00* date_diff( 'minute', a.requested_at, a.completed_at)/60 as diff_hour_req_comp
from moyo_order.mobile_plan_order a --
where 1=1
    and a.order_type is not null -- 예외적으로 없는경우 1건 제외
)
select
*
from order_raw
where 1=1
-- limit 100
"""

df = wr.athena.read_sql_query(sql=sql, database="moyo_mart")
bucket = 'moyo-mart'
table_name = 'order_raw'

res = wr.s3.to_parquet(
    df=df,
    path=f"s3://{bucket}/{table_name}/",
    dataset=True,
    database="moyo_mart",
    table=table_name , 
    mode="overwrite" 
    # partition_cols=['date_ymd']
)